# Hugging Face builder test

In [1]:
# Get the current script directory, from the notebook
import os
notebook_dir = os.getcwd()
print("Notebook directory:", notebook_dir)

model_filename = "v7-1B4"
model_file = os.path.join(notebook_dir, ".model", f"{model_filename}.pth")
print("Model file path:", model_file)

# Check if the model file exists
if os.path.isfile(model_file) is False:
    raise Exception("Model file does not exist")

# Get the project directory two levels up
project_dir = os.path.dirname(os.path.dirname(notebook_dir))
print("Project directory:", project_dir)

# Output build directory
output_dir = os.path.join(notebook_dir, f".hf_build/{model_filename}/")
print("Output directory:", output_dir)

Notebook directory: /home/recursal/rwkv-prj/RWKV-block/test/v7_goose
Model file path: /home/recursal/rwkv-prj/RWKV-block/test/v7_goose/.model/v7-1B4.pth
Project directory: /home/recursal/rwkv-prj/RWKV-block
Output directory: /home/recursal/rwkv-prj/RWKV-block/test/v7_goose/.hf_build/v7-1B4/


In [2]:
# Empty the output directory, if it exists
if os.path.isdir(output_dir):
    import shutil
    print("Removing existing output directory")
    shutil.rmtree(output_dir)
    
# Run the hf_builder.py
!python3 "$project_dir/hf_builder/hf_builder.py" --model_class "v7_goose" "$model_file" "$output_dir"

Removing existing output directory
-----------------------------
Converting RWKV model to HuggingFace format...
Model Class     : v7_goose
Model Source    : /home/recursal/rwkv-prj/RWKV-block/test/v7_goose/.model/v7-1B4.pth
Tokenizer Type  : auto
Output Directory: /home/recursal/rwkv-prj/RWKV-block/test/v7_goose/.hf_build/v7-1B4/
-----------------------------
Building rwkv_block into HF code ...
Loading model weights raw state ...
Loading model config from weights ...
-----------------------------
Model Configuration:
{'vocab_size': 50304, 'num_hidden_layers': 24, 'hidden_size': 2048, 'hidden_size_att': 2048, 'hidden_size_ffn': 8192, 'head_size': 64, 'tmix_backend': 'auto', 'init_state_wkv': False, 'dropout_rate': 0.0, 'use_cache': True, 'return_dict': True, 'output_hidden_states': False, 'output_attentions': False, 'torchscript': False, 'torch_dtype': None, 'use_bfloat16': False, 'tf_legacy_loss': False, 'pruned_heads': {}, 'tie_word_embeddings': False, 'chunk_size_feed_forward': 0, '

In [ ]:
# Load the built model, using the transformers library
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
config = AutoConfig.from_pretrained(output_dir, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(output_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(output_dir, trust_remote_code=True, tmix_backend="cuda")
print("Model and tokenizer loaded successfully")

# Move the model to the GPU
RUN_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model.to(RUN_DEVICE)

# Print the device being used
print("Running on device:", RUN_DEVICE)

# Lets generate some text, using the model on the GPU
prompt = "HELLO WORLD"
print(f"Prompt: {prompt}")

inputs = tokenizer(prompt, return_tensors="pt").to(RUN_DEVICE)
outputs = model.generate(**inputs)
print("Generated text:", tokenizer.decode(outputs[0], skip_special_tokens=True))

/home/recursal/miniconda3/envs/py-3-12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model and tokenizer loaded successfully


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Running on device: cuda
Prompt: HELLO WORLD
Generated text: HELLO WORLD

I'm a writer, a teacher, a mother, a wife, a daughter, a
